<a href="https://colab.research.google.com/github/FrancisKurian/CS530/blob/main/digit_recognizer_computer_vision_ensembles_stacking_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW9
## Summary of steps

*   Tested Random Forest as a baseline model with parameter tuning
*   GridSearch to find l1 and l2 regularization for logistics regression
*   K Neighbhor Classifier - Regularization
*   Super Learner Ensembles(mlens) & SK Learn StackingClassifier
** Evaluate diffrent classification models 
** Creation of SuperLearner using mlens and StackingClassifier
*   Using the best model to predict Kaggle test data.

## Performance Summary:
| Method |Accuracy|            
| :- | :-:|
| Logistics Regression-L1 regularization|91.283%|
| Logistics Regression-L2 regularization|91.762% |
| Random Forest|95.345%|  
| KNeighborsClassifier|96.726%|  
| AdaBoostClassifier|72.929%|
| BaggingClassifier|93.429%|
|Super Learner(mlens)|96.3%|
|SKLearn Stacking Classifier|94.6%

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc,matthews_corrcoef
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from matplotlib import pyplot as plt

In [ ]:
df_train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df_test  = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

X_train, X_test, y_train, y_test = train_test_split(df_train.drop(['label'], axis=1), df_train['label'], test_size=0.2, random_state=88)
print(X_train.shape, y_train.shape,X_test.shape, y_test.shape)

### Testing the RF Classifier on the given data

In [ ]:
rf = RandomForestClassifier(max_depth=30,n_estimators=100, min_samples_leaf=2,min_samples_split=30,criterion='gini', oob_score=True,random_state=42)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)
mc_sk_income_rf=matthews_corrcoef(y_test, y_predict)

probs = rf.predict_proba(X_test)[:,1]

print(f'Matthews Coefficient for SKLearn Random Forest: {mc_sk_income_rf:.5}')
print(f'Accuracy of SKLearn Random Forest: {rf.score(X_test, y_test):.5}')

In [ ]:
plt.hist(y_predict, 
         label='Prediction',alpha=0.5,bins=10)
  
plt.hist(y_test, 
         label='Actual',alpha=0.5,bins=10)
  
plt.legend(loc='upper right')
plt.xticks([*range(0, 10)])
plt.show()

In [ ]:
X=X_train.sample(frac=0.1, replace=False, random_state=1)
y=y_train.sample(frac=0.1, replace=False, random_state=1)

## Logistics Regression- GridSearch for Regularization

In [ ]:
logModel = LogisticRegression()
def hsearch (penalty,solver,iter, l1_ratio,cv):
  param_grid = [    
    {'penalty' : [penalty],
    'C' : [0.00001,0.0001,0.001,0.01,0.1,1],
    'solver' : [solver],
    'max_iter' : [iter],
    'l1_ratio':[l1_ratio]
    }
]
  clf = GridSearchCV(logModel, param_grid = param_grid, cv = cv, scoring='accuracy', verbose=True, n_jobs=-1)
  best_clf = clf.fit(X,y)
  print(best_clf.best_estimator_)
  print (f'Accuracy - : {best_clf.score(X_test,y_test):.3f}')
  return best_clf.best_estimator_

In [ ]:
# be_l1=hsearch(penalty='l1',solver='liblinear',iter=1000, l1_ratio=0.5,cv=5)
# be_l2=hsearch(penalty='l2',solver='sag',iter=3000, l1_ratio=0.5,cv=2)
# be_e=hsearch(penalty='elasticnet',solver='saga',iter=1000, l1_ratio=0.2,cv=2)

print(be_l1,be_l2)

## L1 Lasso Logistics Regression

In [ ]:
clf = be_l1.fit(X_train, y_train)
y_pred=clf.predict(X_test) 
ac_lr=clf.score(X_test, y_test)
mc_lr=matthews_corrcoef(y_test, y_pred)
print(f'The accuracy of Logistic Regression with Lasso(L1/Test) is: {ac_lr:.5}')
print(f'Mattews Correlation Coefficient is: {mc_lr:.5}')

## L2 Ridge Logistics Regression

In [ ]:
clf = be_l2.fit(X_train, y_train)
y_pred=clf.predict(X_test) 
ac_lr=clf.score(X_test, y_test)
mc_lr=matthews_corrcoef(y_test, y_pred)
print(f'The accuracy of Logistic Regression with Ridge(L2/Test) is: {ac_lr:.5}')
print(f'Mattews Correlation Coefficient is: {mc_lr:.5}')
# print(be_l2)

## K Neighbhor Classifier - Regularization

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

k_range =range(1,6)
scores =[]
for k in k_range:
    knn=KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    scores.append(metrics.accuracy_score(y_test, y_pred))
plt.plot(k_range,scores)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=4).fit(X_train,y_train)
y_pred=clf.predict(X_test) 
ac_nb=clf.score(X_test, y_test)
mc_nb=matthews_corrcoef(y_test, y_pred)
print(f'The accuracy of KNeighborsClassifier: {ac_nb:.5}')
print(f'Mattews Correlation Coefficient is: {mc_nb:.5}')

## Prepare the first submission based on the best base model so far

In [ ]:
# Predicting the Test set results
knn=KNeighborsClassifier(n_neighbors=4).fit(X_train,y_train)
y_pred = knn.predict(df_test)
prediction = pd.DataFrame(data = y_pred, columns = ['Label'])
prediction.insert(0, 'ImageId', df_test.index+1)
prediction.head(5)
#Creating a submission
prediction.to_csv(path_or_buf = 'submission.csv', sep = ',', index = False, header = True)

## Super Learner Ensembles(mlens)
### Sampling training dataset to run various algorithms fast

In [ ]:
df_s =df_train.sample(frac=0.2, replace=False, random_state=1)
cols= ["label"]
X1=df_s.drop(columns=cols)
y1=df_s["label"]

In [ ]:
models = list()
models.append(LogisticRegression(C=0.01, max_iter=1000, penalty='l1',solver='liblinear'))
models.append(LogisticRegression(C=1e-05, l1_ratio=0.5,penalty='l2', max_iter=3000, solver='sag'))
models.append(RandomForestClassifier(max_depth=30,n_estimators=100, min_samples_leaf=2,min_samples_split=30,criterion='gini', oob_score=True,random_state=42))
models.append(KNeighborsClassifier(n_neighbors=4))
models.append(AdaBoostClassifier())
models.append(BaggingClassifier(n_estimators=10))

In [ ]:
def evaluate_models(X, y, models,X_test,y_test):
    for model in models:
        clf=model.fit(X,y)
        y_pred=clf.predict(X_test) 
        acc = accuracy_score(y_test,  y_pred)
        print('%s: %.3f' % (model.__class__.__name__, acc*100))

In [ ]:
evaluate_models(X_train,y_train,models,X_test,y_test)

In [ ]:
from mlens.ensemble import SuperLearner
ensemble = SuperLearner(scorer=accuracy_score,folds=10)
ensemble.add(models)
ensemble.add_meta(KNeighborsClassifier(n_neighbors=4))
ensemble.fit(X_train, y_train)
preds = ensemble.predict(X_test)
mc_lr=matthews_corrcoef(y_test, preds)
print(f'Mattews Correlation Coefficient is: {mc_lr:.5}')
print("Prediction score: %.3f" % accuracy_score(preds, y_test))

In [ ]:
# Predicting using the Best base model
y_pred = ensemble.predict(df_test)
prediction = pd.DataFrame(data = y_pred.astype(int), columns = ['Label'])
prediction.insert(0, 'ImageId', df_test.index+1)
prediction.head(5)

In [ ]:
prediction.to_csv(path_or_buf = 'submission.csv', sep = ',', index = False, header = True)

### SK Learn StackingClassifier

In [ ]:
from sklearn.ensemble import StackingClassifier

# Create Base Learners
base_learners = [
                 ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
                 ('bag', BaggingClassifier(n_estimators=10),('knn',KNeighborsClassifier(n_neighbors=4)),('log',LogisticRegression(C=0.01, max_iter=1000, penalty='l1',solver='liblinear')))             
                ]

# Initialize Stacking Classifier with the Meta Learner
clf = StackingClassifier(estimators=base_learners, final_estimator=KNeighborsClassifier(n_neighbors=4))
clf.fit(X_train, y_train).score(X_test, y_test)

## Summary of Classification :
| Method |Accuracy|            
| :- | :-:|
| Logistics Regression-L1 regularization|91.283%|
| Logistics Regression-L2 regularization|91.762% |
| Random Forest|95.345%|  
| KNeighborsClassifier|96.726%|  
| AdaBoostClassifier|72.929%|
| BaggingClassifier|93.429%|
|Super Learner(mlens)|96.3%|
|SKLearn Stacking Classifier|94.6%